## 5-4. 로지스틱 회귀분석을 이용한 문서 분류
로지스틱 회귀분석은 회귀분석에 기반한 분류 알고리즘으로, 회귀분석은 예측하고자 하는 값이 연속적일 때 사용하는 반면 로지스틱 회귀분석은 라벨이 연속적인 값이 아니고 분류(class)에 해당할 때 사용한다. 분류가 이진(binary), 즉 두 개일 때와 셋 이상의 다중 클래스(multi-class)일 때는 서로 다른 알고리즘을 사용하는데, 사이킷런을 이용한다면 그냥 구분 없이 사용하면 된다. 우리가 다루고 있는 문제에서는 학습할 분류의 수가 총 4개이므로 다중 클래스에 해당한다. 

사이킷런은 [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) 클래스를 제공하며, 사이킷런을 사용할 때 좋은 점은 클래스의 사용법이 통일돼 있다는 점이다. 객체를 선언하고, fit() 메서드를 이용해 학습하고, score()를 이용해 정확도를 보거나, predict()로 예측하면 된다. 내부를 들여다보고 싶다면 coef_ 속성을 보면 된다.

In [1]:
% store -r X_train
% store -r X_train_tfidf
% store -r X_test_tfidf
% store -r y_train
% store -r y_test
% store -r tfidf
% store -r newsgroups_train

In [2]:
# sklearn이 제공하는 logistic regression을 사용
from sklearn.linear_model import LogisticRegression

# count vector에 대해 regression을 해서 NB와 비교
LR_clf = LogisticRegression() # 분류기 선언

# train data를 이용해 분류기를 학습
LR_clf.fit(X_train_tfidf, y_train)

# train data에 대한 예측 정확도
print(f"Train set score: {LR_clf.score(X_train_tfidf, y_train):.3f}")
# test data에 대한 예측 정확도
print(f"Test set score: {LR_clf.score(X_test_tfidf, y_test):.3f}")

Train set score: 0.930
Test set score: 0.734


위 결과를 보면 나이브 베이즈보다 성능이 떨어지는 것을 볼 수 있다. 여기에는 여러 가지 이유가 있는데, 첫째로 나이브 베이즈의 가정이 텍스트 분류의 환경과 잘 맞아서 나이브 베이즈가 일반적으로 텍스트 분류에서 놀라울 정도로 좋은 성능을 보여주기 때문이다. 뚤째는 이유리가보다 하나의 가능성인데, 두 알고리즘의 성능을 비교해보면, 학습 데이터와 평가 데이터에 대한 성능 차이가 로지스틱 회귀분석에서 더 큰 것을 볼 수 있다. 이것은 로지스틱 회귀분석에서 과적합이 일어났을 가능성이 있음을 의마한다. 이러한 과적합을 방지하는 방법으로 특성의 수를 줄이는 것과 정규화를 이용해 각 특성에 대한 계수가 지나치게 커지는 것을 방지하는 방법이 있다. 현재 특성의 수는 2,000개로 학습 데이터의 수(2,034개)에 비해 확실히 많은 감이 있다. 그러나 특이하게도 텍스트 분석에서는 특성이 많음에도 불구하고 좋은 성능을 보이는 경우가 많다. 그러니 정규화에 대해 더 알아보자.

### 릿지 회귀를 이용한 과적합 방지
'릿지 회귀(ridge regression)'는 회귀분석에 정규화를 사용하는 알고리즘으로, 앞서 말한 바와 같이 최적화를 위한 목적함수에 정규화 항목을 넣어서 특성에 대한 계수가 지나치게 커지는 것을 억제한다. 이를 직관적으로 설명해보면, 간단한 선으로 주어진 데이터를 설명할 수 있음에도 훨씬 복잡한 선으로 피팅돼 있는 경우, 보통 이렇게 되면 학습 데이터가 아닌 평가 데이터에 대해서는 성능이 급격히 떨어지는 과적합 현상이 발생한다. 기울기가 급격히 변하는 곡선의 경우 특성의 계수가 비교적 크다는 것을 의미하는데, 이와 같은 현상을 방지하려면 곡선이 지나치게 복잡해지지 않도록, 즉 계수가 지나치게 커지지 않도록 계수에 제약을 준다. 이러한 방식의 대표적인 알고리즘이 릿지 회귀이다.

사이킷런에서는 릿지 회귀를 사용하는 분류기로 [RidgeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeClassifier.html) 클래스를 제공한다. RidgeClassifier의 중요한 매개변수로는 alpha가 있는데, alpha는 정규화의 정도를 조절한다. 즉, alpha가 커질수록 정규화의 비중이 커져서 계수의 변화를 더 많이 억제한다. alpha가 너무 커지면 학습 자체가 잘 안되므로 적절한 값을 찾는 것이 중요하다. 이러한 매개변수를 '**하이퍼 파라미터**'라고 하는데, 하이퍼 파라미터는 학습을 통해 정해지는 값이 아니라 사용자가 결정하는 매개변수 값을 말한다. 다음 예에서는 일단 모형이 제공하는 기본값으로 릿지 회귀분석을 실시하고 결과를 본다.

In [3]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier() # 릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) # 학습

print(f"Train set score: {ridge_clf.score(X_train_tfidf, y_train):.3f}")
print(f"Test set score: {ridge_clf.score(X_test_tfidf, y_test):.3f}")

Train set score: 0.960
Test set score: 0.735


테스트 셋에 대한 성능이 아주 조금 나아지기는 했지만, 뭔가 개운하지 못하다. 학습 데이터에 대한 성능이 오히려 올라가서 과적합이 더 심해진 것처럼 보이기 때문이다. alpha를 조정하면 성능을 개선할 수 있을지도 모른다. 적절한 alpha를 찾기 위해서는 학습 데이터를 다시 분리해서 검증(validation) 데이터셋을 만들고, 검증 데이터셋에 대한 성능이 최고가 되는 alpha를 선택해야 한다. 평가 데이터셋을 사용하지 않는 이유는, 평가 데이터셋은 오직 최종적으로 일반화 성능을 평가할 때에만 사용하기 때문이다. 만일 하이퍼 파라미터 값을 찾는 과정에서 평가 데이터셋을 사용하게 되면 일종의 치팅을 한 것이 된다.

아래 코드는 아주 원초적으로 그리드 서치를 구현한 코드다. 그리드 서치는 다양한 하이퍼 파라미터 값에 대해 검증 데이터셋에 대한 성능을 계산해 비교함으로써 적절한 하이퍼 파라미터 값을 얻기 위한 방법이다. 먼저 검증 데이터셋을 얻기 위해 사이킷런이 제공하는 [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)을 이용해 학습데이터를 다시 분리한다. 그 후, alpha 값을 0.1부터 10까지 0.1씩 증가시키면서 릿지 회귀분석을 실시하고 성능이 가장 큰 모형의 alpha를 찾는다. 보통 그리드 서치에서 alpha는 로그스케일, 즉 0.001, 0.01, 0.1, 1, 10, 100과 같은 일련의 값을 사용한다. 여기서는 한 번의 그리드 서치를 한 후에 0.1과 10 사이에서 세밀한 값을 찾는다.

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train_ridge, X_val_ridge, y_train_ridge, y_val_ridge = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)

max_score = 0
max_alpha = 0
for alpha in np.arange(0.1, 10, 0.1): # alpha를 0.1부터 10까지 0.1씩 증가
    ridge_clf = RidgeClassifier(alpha=alpha) # 릿지 분류기 선언
    ridge_clf.fit(X_train_ridge, y_train_ridge) # 학습
    # 검정 데이터셋에 대해 정확도를 측정
    score = ridge_clf.score(X_val_ridge, y_val_ridge)
    if score > max_score: # 정확도가 이전의 정확도 최댓값보다 크면 최댓값을 변경
        max_score = score
        max_alpha = alpha

print(f"Max alpha {max_alpha:.3f} at max validation score {max_score:.3f}")

Max alpha 1.600 at max validation score 0.826


위 예시에서는 최적의 alpha가 1.6으로 나왔으므로 다음과 같이 학습해서 테스트 데이터에 대해 최종 정확도를 계산한다.

In [5]:
ridge_clf = RidgeClassifier(alpha=1.6) # 릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) # 학습

print(f"Train set score: {ridge_clf.score(X_train_tfidf, y_train):.3f}")
print(f"Test set score: {ridge_clf.score(X_test_tfidf, y_test):.3f}")

Train set score: 0.948
Test set score: 0.739


위 결과는 기존의 로지스틱 회귀분석보다는 좋은 성능이지만, 나이브 베이즈에는 못 미치는 것으로 보인다. 이러한 모형 간의 차이는 주어진 데이터와 상황에 따라 달라서 일반화하기는 어렵지만, 적어도 나이브 베이즈가 무시 못할 성능을 보이는 것은 알 수 있다.

나이브 베이즈에서 우리가 분류에 중요한 영향을 미친 특성, 즉 단어들을 살펴본 것과 같이 릿지 회귀분석에서도 단어들을 분석할 수 있다. 함수화한 top10_features()를 이용해 한번 결과를 살펴보자.

In [6]:
def top10_features(classifier, vectorizer, categories):
    feature_names = np.asarray(vectorizer.get_feature_names_out())
    for i, category in enumerate(categories):
        top10 = np.argsort(-classifier.coef_[i])[:10]
        print(f'{category}: {", ".join(feature_names[top10])}')

top10_features(ridge_clf, tfidf, newsgroups_train.target_names)

alt.atheism: bobby, religion, atheism, atheists, motto, punishment, islam, deletion, islamic, satan
comp.graphics: graphics, computer, 3d, file, image, hi, 42, using, screen, looking
sci.space: space, orbit, nasa, spacecraft, moon, sci, launch, flight, funding, idea
talk.religion.misc: christian, christians, fbi, blood, order, jesus, objective, children, christ, hudson


나이브 베이즈가 you, are, not, be, this 같은 일상적인 단어들을 보여준 것에 비해 릿지 회귀는 훨씬 그럴 듯한 결과를 보여준다. 바로 이러한 점이 로지스틱 회귀분석 계열의 장점이다. 이러한 분석을 통해 예측 결과를 설명하고 해석하기에 매우 용이하기 때문이다. 회귀분석에 정규화를 추가하는 알고리즘으로 릿지 회귀분석 외에 라쏘 회귀분석이 있다. 라쏘 회귀분석은 특성을 골라낼 수 있도록 하는 장점이 있다.

### 라쏘 회귀를 이용한 특성 선택
'라쏘 회귀(lasso regression)는 특성의 개수에 대해 정규화를 한다는 점에서는 유사하지만 정규화 항에 차이가 있다. 릿지는 L2 정규화를 쓰는 반면, 라쏘는 L1 정규화를 사용한다. 정규화 항에 결과적으로 어떤 차이가 있는지를 설명하자면, 라쏘는 정규화를 할 때 특성의 계수가 0에 가까워지면 이를 완전히 0으로 바꾼다는 점에서 차이가 있다. 어떤 특성의 계수가 0이라는 것은 그 특성은 실제로 분석에 전혀 영향을 미치지 않으며 사실상 그 특성 값이 사용되지 않는다는 것을 의미한다. 이는 특성의 수를 줄이는 결과를 가져온다. 보통 특성의 수가 줄어들수록 정확도도 함께 줄어드는 경향이 있다. 쉽게 생각해서 정보량이 줄어든다고 봐도 좋다. 라쏘는 정규화를 통해 과적합을 줄이지만, 동시에 특성의 수도 줄어드므로 성능, 즉 정확도는 항상 향상된다고 보기 어렵다. 다양한 시도를 통해 적절한 지점을 찾는 것이 중요하다.

사이킷런은 라쏘 회귀분석을 사용하는 별도으 분류기를 제공하지 않고 기존의 로지스틱 회귀분석에서 [정규화 방식을 L1으로 선택](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)하도록 함으로써 지원한다. 동시에 알고리즘도 'liblinear'를 선택해야 한다. 그리고 릿지 회귀의 alpha와 같은 기능을 하는 매개변수로 C를 제공하는데, 주의할 점은 C가 alpha의 역수라는 것이다. 즉, 릿지 회귀에서는 alpha를 올리면 정규화가 강해지지만 C는 값이 커지면 정규화가 약해진다. 아래 예시는 C에 1을 인수로 주고 라쏘 회귀분석을 이용하는 분류를 하는 예다.

In [7]:
# Lasso는 동일한 LogisticRegression을 사용하면서 매개변수로 지정
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear', C=1)

lasso_clf.fit(X_train_tfidf, y_train) # train data로 학습

print(f"# Train set score: {lasso_clf.score(X_train_tfidf, y_train):.3f}")
print(f"# Test set score: {lasso_clf.score(X_test_tfidf, y_test):.3f}")

# 계수(coefficient) 중에서 0이 아닌 것들의 개수를 출력
print(f"# Used features count: {np.sum(lasso_clf.coef_ != 0)} out of {X_train_tfidf.shape[1]}")

# Train set score: 0.819
# Test set score: 0.724
# Used features count: 437 out of 2000


위 결과를 보면 정확도는 떨어졌고, 사용된 특성은 전체 2,000개 중에서 437개로 줄어든 것을 볼 수 있다. 라쏘와 같은 방식으로 특성의 수를 줄이는 것을 '**특성 선택(feature selection)**'이라고 한다. 특성을 줄이면서 얻어지는 장점은 연관성이 떨어지는 특성들을 배제하고 중요한 특성에 집중할 수 있다는 것이다. 앞에서 한 것처럼 카테고리별로 상위 10개의 중요 특성들을 살펴보자.

In [8]:
top10_features(lasso_clf, tfidf, newsgroups_train.target_names)

alt.atheism: bobby, atheism, atheists, islam, religion, islamic, motto, atheist, satan, vice
comp.graphics: graphics, image, 3d, file, computer, hi, video, files, looking, sphere
sci.space: space, orbit, launch, nasa, spacecraft, flight, moon, dc, shuttle, solar
talk.religion.misc: fbi, christian, christians, christ, order, jesus, children, objective, context, blood


해석하는 사람이나 상황에 따라 달라질 수 있으나, 보기에 따라서는 릿지 회귀분석보다 이 단어들이 각 카테고리를 더 잘 반영한다고 볼 수 있다. 라쏘는 성능을 향상할 목적으로도 사용되지만 특성의 수를 줄이고 특성에 대한 설명을 더 잘하기 위해서도 많이 사용된다.